In [3]:
!pip install langchain openai tiktoken



In [4]:
import inspect

from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [5]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [6]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [9]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [11]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!'),
  AIMessage(content="what's up?")]}

In [12]:
import os
os.environ["OPENAI_API_KEY"] = "your api key"

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [14]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


{'question': 'hi',
 'chat_history': '',
 'text': ' Hello! How are you doing today?'}

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [16]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I help you today?')],
 'text': 'Hello! How can I help you today?'}

In [17]:
from langchain.chains.conversation.memory import ConversationBufferMemory
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [18]:
conversation = ConversationChain(
    llm=llm,
)

In [19]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [20]:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [21]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [22]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

In [23]:
conversation_buf("Can you please tell me your name AI?")

{'input': 'Can you please tell me your name AI?',
 'history': 'Human: Good morning AI!\nAI: Good morning! How can I assist you today?',
 'response': 'My name is OpenAI. I am an AI developed by OpenAI, a leading research organization in artificial intelligence. How may I help you today?'}

In [24]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [25]:
count_tokens(
    conversation_buf,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)


Spent a total of 271 tokens


'That sounds interesting! The concept of a Big Hairy Audacious Goal (BHAG) was first introduced by Jim Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." A BHAG is a powerful tool that organizations can use to set a long-term vision and inspire their employees and stakeholders. It is meant to be ambitious and daring, pushing the boundaries of what seems possible. By having a BHAG, companies can rally their team around a common purpose and create a sense of shared mission.'

In [26]:
count_tokens(
    conversation_buf,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)

Spent a total of 424 tokens


'Absolutely! Embracing a BHAG can indeed help an organization maintain focus on a long-term goal and create a sense of urgency. It provides a clear direction and purpose for everyone involved, which can be a powerful motivator. Additionally, having a BHAG can make an organization more attractive to top talent, as it signifies ambition and a commitment to achieving something truly meaningful. By aligning the efforts of employees and stakeholders towards a BHAG, organizations can galvanize their teams and increase their chances of success.'

In [27]:
count_tokens(
    conversation_buf,
    "What concept this book introduces?"
)

Spent a total of 451 tokens


'This book introduces the concept of a Big Hairy Audacious Goal (BHAG).'

In [28]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: Can you please tell me your name AI?
AI: My name is OpenAI. I am an AI developed by OpenAI, a leading research organization in artificial intelligence. How may I help you today?
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI: That sounds interesting! The concept of a Big Hairy Audacious Goal (BHAG) was first introduced by Jim Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." A BHAG is a powerful tool that organizations can use to set a long-term vision and inspire their employees and stakeholders. It is meant to be ambitious and daring, pushing the boundaries of what seems possible. By having a BHAG, companies can rally their team around a common purpose and create a sense of shared mission.
Human: The BHAG can he

In [29]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)


In [30]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [31]:
# without count_tokens we'd call `conversation_sum("Can you please tell me your name AI?!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Can you please tell me your name AI?"
)

Spent a total of 344 tokens


'I am an AI developed by OpenAI, known as GPT-3. My purpose is to assist and engage in conversations with users like yourself. How can I assist you today?'

In [32]:
count_tokens(
    conversation_sum,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)

Spent a total of 894 tokens


'That sounds interesting! The concept of a Big Hairy Audacious Goal (BHAG) was actually popularized by Jim Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." They define a BHAG as a long-term, ambitious, and inspiring goal that can rally people together and drive organizational success. It\'s meant to be a clear and compelling vision that stretches and challenges the organization, while also being achievable. BHAGs are often used by companies to create a sense of purpose and direction, and they can be a powerful tool for motivating employees and aligning their efforts towards a common objective. Is there anything specific you would like to know about BHAGs or how they can be implemented?'

In [33]:
count_tokens(
    conversation_sum,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)


Spent a total of 1262 tokens


'That\'s absolutely right! Embracing a BHAG can indeed help an organization focus on a long-term goal and create a sense of urgency. It provides a clear direction and purpose for the organization, which can motivate employees and align their efforts. By setting a big, ambitious goal, companies can attract and retain top talent who are excited to be part of something meaningful and impactful.\n\nThe book "Built to Last: Successful Habits of Visionary Companies" by Jim Collins and Jerry Porras explores the concept of BHAGs in depth. It showcases how successful companies have used BHAGs to drive their success and sustain their competitive advantage over time. The authors emphasize the importance of having an audacious goal that inspires and stretches the organization, rather than settling for mediocrity.\n\nIf you have any more questions about BHAGs or their implementation, feel free to ask!'

In [34]:
count_tokens(
    conversation_sum,
    "What concept this book introduces?"
)

Spent a total of 910 tokens


'The book "Built to Last: Successful Habits of Visionary Companies" introduces the concept of BHAGs, which stands for "Big Hairy Audacious Goals." BHAGs are long-term, ambitious goals that can motivate and align employees. They are meant to be challenging and exciting, and they can help organizations focus, create a sense of urgency, and attract top talent. The book explores how successful companies have used BHAGs to drive their success and provides insights into the habits and practices of these visionary companies.'

In [35]:
count_tokens(
    conversation_sum,
    "the book encourages readers of what?"
)

Spent a total of 702 tokens


'The book "Built to Last: Successful Habits of Visionary Companies" encourages readers to adopt and implement the concept of BHAGs, which stands for "Big Hairy Audacious Goals." BHAGs are long-term, ambitious goals that can motivate and align employees. The book emphasizes the importance of setting challenging and exciting goals that can help organizations focus, create a sense of urgency, and attract top talent.'

In [36]:
print(conversation_sum.memory.buffer)

The AI explains that the book "Built to Last: Successful Habits of Visionary Companies" encourages readers to adopt and implement the concept of BHAGs, which are long-term, ambitious goals that can motivate and align employees. BHAGs are meant to be challenging and exciting, and they can help organizations focus, create a sense of urgency, and attract top talent. The book explores how successful companies have used BHAGs to drive their success and provides insights into the habits and practices of these visionary companies.


In [37]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory



In [38]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

In [39]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 77 tokens


'Good morning! How can I assist you today?'

In [41]:
count_tokens(
    conversation_bufw,
    "I want to know more about Singularity, what it is and when it is coming."
)


Spent a total of 301 tokens


'Singularity refers to a hypothetical future point in time when technological growth becomes uncontrollable and irreversible, resulting in unforeseeable changes to human civilization. It is often associated with the emergence of superintelligent machines that surpass human intelligence in every aspect. The concept was popularized by mathematician and computer scientist Vernor Vinge in the 1990s and further explored by futurist Ray Kurzweil.\n\nAs for when Singularity might occur, it is difficult to predict with certainty. Some experts believe it could happen within the next few decades, while others argue that it is still far off in the future. The development and arrival of superintelligent AI systems are dependent on various factors, including advancements in technology, algorithms, and computational power.\n\nIt is worth noting that the concept of Singularity is highly debated, and there are different opinions regarding its feasibility and potential impact. Some see it as a potentia

In [42]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 323 tokens


'Please let me know what specific information you would like to know about Singularity, and I will do my best to provide you with the requested details.'

In [43]:
count_tokens(
    conversation_bufw,
    "Give me the technology behind it"
)


Spent a total of 449 tokens


'The concept of Singularity refers to a hypothetical point in the future where artificial intelligence (AI) surpasses human intelligence, leading to rapid technological advancements that are beyond our current comprehension. While the idea of Singularity encompasses various aspects, there are several key technologies that are often associated with its potential realization.\n\nOne of the fundamental technologies behind Singularity is artificial general intelligence (AGI). AGI refers to highly autonomous systems that possess the ability to outperform humans in most economically valuable work. It involves creating AI systems that can understand, learn, and apply knowledge across a wide range of tasks and domains. AGI is an essential component for achieving Singularity since it requires machines that are capable of surpassing human intellectual capabilities.\n\nAnother technology that plays a crucial role in Singularity is machine learning. Machine learning involves developing algorithms 

In [44]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 468 tokens


'I apologize, but I do not have the ability to know what specific information you want to know without you telling me. Please provide me with your question or topic of interest, and I will do my best to provide you with the relevant information.'

In [45]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

In [46]:
conversation_sum_bufw = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
    ),
)


In [47]:
print(conversation_sum_bufw.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
